In [1]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score
import sklearn.preprocessing as pre
import json
import pandas as pd
import numpy as np
import sklearn.model_selection
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest
import sklearn
import tensorflow.compat.v1 as tf
import tensorflow
import tensorflow.keras as keras
import warnings
warnings.filterwarnings('ignore')
tf.disable_v2_behavior()
from sklearn.base import BaseEstimator,ClassifierMixin

data_dir = '../../data/processed/'

Instructions for updating:
non-resource variables are not supported in the long term


baseline:   


    1. use random search for hyper-parameter
    2. KS值作为最重要的指标， the best threshold for the classifier is based on KS
    3. Done: SVM, Lasso, logistic
    4. with probability: Logistic, NN
    5. without probability: Tree. 
        XGBoost, Random Forest
    6. try with bagging. 
    7. use model: ensemble model
        ereg = VotingRegressor(estimators=[('gb', reg1), ('rf', reg2), ('lr', reg3)])
        reg1 = GradientBoostingRegressor(random_state=1, n_estimators=10)
        reg2 = RandomForestRegressor(random_state=1, n_estimators=10)
    8. naive bayes

        


# preparing the data

## adding the crime data into it

In [2]:
# X=pd.read_csv()
with open(data_dir+"non_sequential_features.json") as obj:
    X = json.load(obj)
with open(data_dir+"padded_sequential_features_3.json") as obj:
    X_seq = json.load(obj)
with open(data_dir+"label.json") as obj:
    Y = json.load(obj)

In [3]:
feature2=np.array([X_seq[key] for key in X_seq.keys()])

In [4]:
feature2.shape

(3715, 60, 18)

In [5]:
feature2 = np.concatenate(
    [feature2[:, :, 0][:, :, np.newaxis], feature2[:, :, 14:]], axis=-1)

In [6]:
feature2 = feature2.reshape((-1, 60*5))

In [7]:
X = np.array([X[key] for key in X.keys()])

In [8]:
X = np.concatenate([X, feature2], axis=-1)

In [9]:
Y_ = Y.values()

In [10]:
Y_ = np.array(list(Y_))

In [11]:
Y_ = Y_.astype("int")

In [12]:
X_ = X

In [13]:
X_.shape, Y_.shape

((3715, 314), (3715,))

In [44]:
index = np.arange(len(X_))
X, X_test_2019, Y, Y_test_2019 = sklearn.model_selection.train_test_split(X_[:100000], Y_[:100000], test_size=0.25)

# parameters

In [15]:
def get_KS(y_prob, y_true):
    '''

    return the best threshold, maximum ks

    '''

    fpr, tpr, threshold = roc_curve(y_true, y_prob)
    ks = (tpr-fpr)
    max_ = np.argmax(ks)

    return threshold[max_], np.max(ks)

# Test on important features

In [16]:
class Get_best(BaseEstimator, ClassifierMixin):
    def __init__(self, model, threshold):
        self.model = model
        self.threshold = threshold

    def fit(self, X, Y):
        self.model.fit(X, Y)
        return self

    def predict_proba(self, X):
        self.proba = self.model.predict_proba(X)
        return self.proba

    def predict(self, X):
        return self.predict_proba(X)[:, 1] >= self.threshold

In [17]:
class selection_and_KS:
    def __init__(self, model, parameter, model_name=None):
        self.model = model
        self.parameter = parameter
        self.scoring = {'AUC': 'roc_auc'}
        self.refit_first = 'AUC'
        self.cv = 5
        self.clf = RandomizedSearchCV(self.model, param_distributions=self.parameter,
                                      cv=self.cv, scoring=self.scoring, refit=self.refit_first)
        self.model_name = model_name

    def fit(self, X, Y):
        if self.model_name == "SVM":
            self.scoring = {'AUC': 'accuracy'}
        self.clf.fit(X, Y)
        self.best_model = self.clf.best_estimator_
        if self.model_name == "SVM":
            self.best_model = self.best_model.set_params(probability=True)
        self.best_model.fit(X, Y)
        self.threshold, self.ks = get_KS(
            self.best_model.predict_proba(X)[:, 1], Y)
        print("ks-value is: {}".format(self.ks))
        self.accuracy = np.mean((self.best_model.predict(X) == Y))
        print(self.clf.cv_results_['params'][self.clf.best_index_])
        return self

    def return_best(self):
        return Get_best(model=self.best_model, threshold=self.threshold)

In [18]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

In [19]:
from sklearn.model_selection import RandomizedSearchCV

# base model

## for voting ensembling

## logistics Ridge

In [20]:
model_dict = {}
model_acc = {}
model_ks = {}
X_dict = {}
Y_dict = {}

## SVM

our dataset is so big that SVM is not appropriate here.

In [21]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

## LDA&QDA

In [22]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis,QuadraticDiscriminantAnalysis

### LDA

In [23]:
LDA = LinearDiscriminantAnalysis()
search_parameter = {'solver': ['svd', 'lsqr']}
m_LDA = selection_and_KS(LDA, parameter=search_parameter)
m_LDA.fit(X, Y)
model_dict["LDA"] = m_LDA.return_best()
model_acc["LDA"] = m_LDA.accuracy
model_ks["LDA"] = m_LDA.ks
X_dict["LDA"] = X
Y_dict["LDA"] = Y

ks-value is: 0.5145390978084762
{'solver': 'svd'}


### QDA

In [24]:
QDA = QuadraticDiscriminantAnalysis()
search_parameter = {}
m_QDA = selection_and_KS(QDA, parameter=search_parameter)
m_QDA.fit(X, Y)
model_dict["QDA"] = m_QDA.return_best()
model_acc["QDA"] = m_QDA.accuracy
model_ks["QDA"] = m_QDA.ks
X_dict["QDA"] = X
Y_dict["QDA"] = Y

ks-value is: 1.0
{}


## Tree model: Random_forest, XGboost

### random_forest

In [25]:
params = {"max_depth": [i for i in range(
    2, 32)], 'max_features': range(3, 11, 2)}
model = RandomForestClassifier()
random_forest = selection_and_KS(model, parameter=params)
random_forest.fit(X, Y)
model_dict["tree_random_forest"] = random_forest.return_best()
model_acc["tree_random_forest"] = random_forest.accuracy
model_ks["tree_random_forest"] = random_forest.ks
X_dict["tree_random_forest"] = X
Y_dict["tree_random_forest"] = Y

ks-value is: 1.0
{'max_features': 9, 'max_depth': 28}


### XGBOOST

In [26]:
from xgboost.sklearn import XGBClassifier

In [27]:
params = {"max_depth": [i for i in [32]],
          'max_features': [16], 'learning_rate': [0.01]}
model = XGBClassifier()
tree_XGBOOST = selection_and_KS(model, parameter=params)
tree_XGBOOST.fit(X, Y)
model_dict["tree_XGBOOST"] = tree_XGBOOST.return_best()
model_acc["tree_XGBOOST"] = tree_XGBOOST.accuracy
model_ks["tree_XGBOOST"] = tree_XGBOOST.ks
X_dict["tree_XGBOOST"] = X
Y_dict["tree_XGBOOST"] = Y

ks-value is: 0.9171825701372901
{'max_features': 16, 'max_depth': 32, 'learning_rate': 0.01}


# soft_voting

In [28]:
class soft_voting:
    def __init__(self,model_dict,X_dict,Y_dict):
        self.model_dict=model_dict
        self.X_dict=X_dict
        self.Y_dict=Y_dict
    def train(self):
        for key in self.model_dict:
            self.model_dict[key].fit(self.X_dict[key],Y_dict[key])
            y_=self.model_dict[key].predict(self.X_dict[key])
            tn, fp, fn, tp = confusion_matrix(Y_dict[key],y_).ravel()
            
            print("in sample for model: {}; fpr={}, tpr={}".format(key,fp/(fp+tn),tp/(tp+fn)))
            
    def predict_proba(self,X_dict,weight,seperate="Tree"):
        non_tree=np.sum([weight[key] for key in weight if "tree" not in key])
        tree=np.sum([weight[key] for key in weight if "tree" in key])
        scaler=tree/(non_tree+1e-11)
        if seperate=='Tree':
            for key in X_dict:
                frame=np.zeros(shape=(X_dict[key].shape[0],2))
#                 print(key)
                if "tree" not in key:
                    frame=frame+self.model_dict[key].predict_proba(X_dict[key])*scaler
                else:
                    frame=frame+self.model_dict[key].predict_proba(X_dict[key])
        else:
            for key in X_dict:
                frame=np.zeros(shape=(X_dict[key].shape[0],2))
                frame=frame+self.model_dict[key].predict_proba(X_dict[key])
        return frame
    def predict(self,X_dict,weight,seperate="Tree"):
        return np.argmax(self.predict_proba(X_dict,weight,seperate),axis=1)
    
    
    def compare(self,X_dict,Y_dict):
        for key in self.model_dict:
            y_=self.model_dict[key].predict(X_dict[key])
            tn, fp, fn, tp = confusion_matrix(Y_dict[key],y_).ravel()
            print("in sample for model: {}; fpr={}, tpr={}, acc={}".format(key,fp/(fp+tn),tp/(tp+fn),np.mean(Y_dict[key]==y_)))
            y_proba=self.model_dict[key].predict_proba(X_dict[key])
            _,ks=get_KS(y_proba[:,1],Y_dict[key])
            print("ks for"+key+"is :"+str(ks))        

In [29]:
final_model=soft_voting(model_dict,X_dict,Y_dict)

In [30]:
final_model.train()

in sample for model: LDA; fpr=0.2632386799693016, tpr=0.7777777777777778
in sample for model: QDA; fpr=0.0, tpr=1.0
in sample for model: tree_random_forest; fpr=0.0, tpr=0.9888888888888889
in sample for model: tree_XGBOOST; fpr=0.03837298541826554, tpr=0.9555555555555556


In [31]:
# X_test_2019,Y_test_2019=X_test_2019[:2000],Y_test_2019[:2000]
# X_test_2019_non,Y_test_2019_non=X_test_2019_non[:2000],Y_test_2019_non[:2000]

In [32]:
Y_test_data={}
X_test_data={}
for i in X_dict:
    if "tree" in i:
        Y_test_data[i]=Y_test_2019
        X_test_data[i]=X_test_2019
    else:
        Y_test_data[i]=Y_test_2019
        X_test_data[i]=X_test_2019

# with the model trained before, lets select

## First lets see in-sample metrcs

### imbalance weight：tree=50%, linear=50%

In [33]:
proba=final_model.predict_proba(X_dict,weight=model_ks)
predict=final_model.predict(X_dict,weight=model_ks)
tn, fp, fn, tp = confusion_matrix(predict,Y).ravel()
print("in sample for model: {}; fpr={}, tpr={}, acc={}".format("voting",fp/(fp+tn),tp/(tp+fn),np.mean(predict==Y)))

in sample for model: voting; fpr=0.03518518518518519, tpr=0.9883720930232558, acc=0.9655419956927495


In [34]:
proba[:,1]

array([0.19244359, 0.24120206, 0.19441538, ..., 0.22799245, 0.20492069,
       0.19328833])

In [35]:
get_KS(proba[:,1],Y)

(0.27197638154029846, 0.9171825701372901)

### balance weight

In [36]:
proba = final_model.predict_proba(X_dict, weight=model_ks, seperate="None")
predict = final_model.predict(X_dict, weight=model_ks, seperate="None")
tn, fp, fn, tp = confusion_matrix(predict, Y).ravel()
print("in sample for model: {}; fpr={}, tpr={}, acc={}".format(
    "voting", fp/(fp+tn), tp/(tp+fn), np.mean(predict == Y)))

in sample for model: voting; fpr=0.03518518518518519, tpr=0.9883720930232558, acc=0.9655419956927495


In [37]:
get_KS(proba[:,1],Y)

(0.27197638154029846, 0.9171825701372901)

### compare

In [38]:
final_model.compare(X_dict,Y_dict)

in sample for model: LDA; fpr=0.2632386799693016, tpr=0.7777777777777778, acc=0.7394113424264178
ks forLDAis :0.5145390978084762
in sample for model: QDA; fpr=0.0, tpr=1.0, acc=1.0
ks forQDAis :1.0
in sample for model: tree_random_forest; fpr=0.0, tpr=0.9888888888888889, acc=0.9992821249102656
ks fortree_random_forestis :1.0
in sample for model: tree_XGBOOST; fpr=0.03837298541826554, tpr=0.9555555555555556, acc=0.9612347451543432
ks fortree_XGBOOSTis :0.9171825701372901


## the test dataset metrics

### imbalance

In [39]:
proba=final_model.predict_proba(X_test_data,weight=model_ks)
predict=final_model.predict(X_test_data,weight=model_ks)
tn, fp, fn, tp = confusion_matrix(predict,Y_test_2019).ravel()
print("out sample for model: {}; fpr={}, tpr={}, acc={}".format("voting",fp/(fp+tn),tp/(tp+fn),np.mean(predict==Y_test_2019)))

out sample for model: voting; fpr=0.07883369330453564, tpr=0.3333333333333333, acc=0.9192680301399354


In [40]:
get_KS(proba[:,1],Y_test_2019)

(0.25037384033203125, 0.17253042516200412)

### balance

In [41]:
proba = final_model.predict_proba(X_test_data, weight=model_ks, seperate="None")
predict = final_model.predict(X_test_data, weight=model_ks, seperate="None")
tn, fp, fn, tp = confusion_matrix(predict, Y_test_2019).ravel()
print("out sample for model: {}; fpr={}, tpr={}, acc={}".format(
    "voting", fp/(fp+tn), tp/(tp+fn), np.mean(predict == Y_test_2019)))

out sample for model: voting; fpr=0.07883369330453564, tpr=0.3333333333333333, acc=0.9192680301399354


In [42]:
get_KS(proba[:,1],Y_test_2019)

(0.25037384033203125, 0.17253042516200412)

### compare

In [43]:
final_model.compare(X_test_data,Y_test_data)

in sample for model: LDA; fpr=0.3146198830409357, tpr=0.32432432432432434, acc=0.6566200215285253
ks forLDAis :0.06867393709498976
in sample for model: QDA; fpr=0.0, tpr=0.0, acc=0.9203444564047363
ks forQDAis :0.0
in sample for model: tree_random_forest; fpr=0.0, tpr=0.0, acc=0.9203444564047363
ks fortree_random_forestis :0.03960802908171329
in sample for model: tree_XGBOOST; fpr=0.1368421052631579, tpr=0.22972972972972974, acc=0.8127018299246501
ks fortree_XGBOOSTis :0.17253042516200412
